Installing Dependencies

In [224]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

Test DATA

In [225]:
# import pandas as pd

# # Sample data
# data = {
#     'speed of operation': [0, 1, 2, 3, 4],
#     'load condition': [0,1,0,0,1],
#     'Speed condition': [1,0,1,0,1],
#     'batch process': [1, 1, 1, 0, 0],
#     'mounting condition': [0, 1, 2, 1, 0],
#     'Infinite Uptime pred': [0,0,1,1,0],
#     'customer feedback':[0,1,0,1,0]
# }

# # Create the DataFrame
# df = pd.DataFrame(data)

In [226]:
import pandas as pd
df = pd.read_excel('C:\\Users\\advan\\OneDrive\\Desktop\\Infinite Uptime\\DATA\\train.xlsx')
df.drop(['plant_name','machine_id','monitor_id'],axis=1,inplace=True)
df

,plant_id,parameter,magnitude,CoE Validation as per SOP,org_name,industry_type,mounting_condition,speed,speed_type,m_category
0,0,0,123.60,0,0,0,1,1482.0,1,1
1,1,1,70.99,0,0,0,1,2940.0,1,1
2,2,2,9.05,0,1,1,1,2950.0,1,2
3,3,2,3.44,1,2,2,0,150.0,2,3
4,4,2,3.15,1,0,0,1,34.0,1,4
...,...,...,...,...,...,...,...,...,...,...
632,78,2,10.30,0,0,0,3,2985.0,1,2
633,24,0,1774.96,0,0,0,3,2925.0,1,1
634,36,2,1.53,1,0,0,0,60.0,1,3
635,10,0,1597.73,0,7,0,1,1490.0,1,1


In [227]:
df_dis = df.drop(columns=['magnitude','speed'])
df_dis

,plant_id,parameter,CoE Validation as per SOP,org_name,industry_type,mounting_condition,speed_type,m_category
0,0,0,0,0,0,1,1,1
1,1,1,0,0,0,1,1,1
2,2,2,0,1,1,1,1,2
3,3,2,1,2,2,0,2,3
4,4,2,1,0,0,1,1,4
...,...,...,...,...,...,...,...,...
632,78,2,0,0,0,3,1,2
633,24,0,0,0,0,3,1,1
634,36,2,1,0,0,0,1,3
635,10,0,0,7,0,1,1,1


In [228]:
df_cont = df[['magnitude','speed']]
# df_cont

In [229]:
MultiDiscrete([5,2,2,2,3,2,2]).sample()

array([2, 1, 0, 0, 1, 1, 1], dtype=int64)

Building Environment

In [230]:
unique_values =df.nunique()
unique_values =unique_values.tolist()
unique_values

[100, 3, 590, 2, 30, 15, 7, 110, 3, 10]

In [231]:
observation_space=Dict({'plant_id':Discrete(2), 'parameter':Discrete(3), "magnitude":Box(0.0,20000.0,shape=(1,), dtype=float),'CoE Validation as per SOP':Discrete(2),'org_name':Discrete(30),
      'industry_type':Discrete(15),'mounting_condition':Discrete(7),"speed":Box(0.0,20000.0,shape=(1,), dtype=float),'speed_type':Discrete(3),'m_category':Discrete(10) })

In [232]:
# discrete_values = df_dis.nunique().tolist()  
# print(discrete_values)

# # Define the box values (continuous values)
# # low = [0.0, 0.0, 0.0]
# # high = [20000.0, 20000.0, 20000.0]

# # Create the individual observation spaces
# discrete_space = MultiDiscrete(discrete_values)
# box_space = Box(0.0,20000.0,shape=(2,), dtype=float)

# # Combine the observation spaces using a Tuple space
# observation_space = Tuple((discrete_space, box_space))

In [233]:
def value(col:str, n:int):
    return df[col].iloc[n]

In [234]:
def dic(n:int):
    empty_dic ={}
    for i,key in enumerate(observation_space.keys()):
        empty_dic[key] = value(key,n)
    return empty_dic

In [235]:
# class TestEnv(Env):
#     def __init__(self):
#         # Actions we can alert(1),no alert(0)
#         self.action_space = Discrete(1)
#         # feature array
#         self.observation_space = observation_space
#         # Set start state
#         self.state = df_dis.iloc[0].tolist()+df_cont.iloc[0].tolist()
        
#         # Set run length
#         self.length = 637

        
#     def step(self, action):
#         self.state=df_dis.iloc[637-self.length].tolist()+df_cont.iloc[637-self.length].tolist()
#         self.length -= 1 
        
#         # Calculate reward
#         if self.state[2] == action :
#             reward =1 
#         else: 
#             reward = -1 
        

#         if self.length <= 0: 
#             done = True
#         else:
#             done = False
        
#         info = {}
        
#         # Return step information
#         return self.state, reward, done, info

#     def render(self):
#         # Implement viz
#         pass
    
#     def reset(self):
#         self.state = np.array(df_dis.iloc[0].tolist()+df_cont.iloc[0].tolist())
#         self.length = 637
#         return self.state

In [236]:
class TestEnv(Env):
    def __init__(self):
        # Actions we can alert(1),no alert(0)
        self.action_space = Discrete(2)
        # feature array
        self.observation_space = observation_space
        # Set start state
        self.state = dic(0)
        
        # Set run length
        self.length = 637

        
    def step(self, action):
        self.state=dic(637-self.length)
        self.length -= 1 
        
        # Calculate reward
        if self.state['CoE Validation as per SOP'] == action :
            reward =1 
        else: 
            reward = -1 
        

        if self.length <= 0: 
            done = True
        else:
            done = False
        
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        self.state = dic(0)
        self.length = 637
        return self.state

In [237]:
env=TestEnv()

In [238]:
env.observation_space.sample()

OrderedDict([('CoE Validation as per SOP', 0),
             ('industry_type', 9),
             ('m_category', 1),
             ('magnitude', array([14751.96712765])),
             ('mounting_condition', 0),
             ('org_name', 0),
             ('parameter', 0),
             ('plant_id', 1),
             ('speed', array([5470.59449384])),
             ('speed_type', 1)])

In [239]:
env.action_space.sample()


0

In [240]:
env.reset()

{'CoE Validation as per SOP': 0,
 'industry_type': 0,
 'm_category': 1,
 'magnitude': 123.6,
 'mounting_condition': 1,
 'org_name': 0,
 'parameter': 0,
 'plant_id': 0,
 'speed': 1482.0,
 'speed_type': 1}

Testing Environment

In [241]:
# from stable_baselines3 import PPO
# from stable_baselines3.common.envs import SimpleMultiObsEnv


# # Stable Baselines provides SimpleMultiObsEnv as an example environment with Dict observations
# env = SimpleMultiObsEnv(random_start=False)



# model = PPO("MultiInputPolicy", env, verbose=1)
# model.learn(total_timesteps=100_000)

In [242]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        # print(action)
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:35
Episode:2 Score:37
Episode:3 Score:-27
Episode:4 Score:-29
Episode:5 Score:25


Training RL model

In [243]:
log_path = os.path.join('Training', 'Logs')
model = PPO("MultiInputPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\advan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [244]:
model.learn(total_timesteps=4000)

Logging to Training\Logs\PPO_7


RuntimeError: Class values must be smaller than num_classes.

In [36]:
model.save(os.path.join('Training','Saved Models','V1'))